<a href="https://colab.research.google.com/github/CamiloVga/Curso-Inteligencia-Artificial/blob/main/Clase_FineTuning_Classifier_PasoAPaso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning para un modelo de clasificación

Explicación paso a paso para crear un modelo Fine-Tuning de un modelo Bert para la clasificación de texto en categorías. Se agrega la opción para crear datos sintéticos.

**Creado por [Camilo Vega](https://www.linkedin.com/in/camilo-vega-169084b1/)**

# Generación datos sintéticos

In [ ]:
# Paso 1: Instalar las librerías necesarias
!pip install openai==0.28
import openai
import pandas as pd

# Paso 2: Configurar la clave de API de OpenAI
# Reemplaza 'SU_CLAVE_DE_API' con tu clave de API de OpenAI
openai.api_key = 'API KEY'

# Paso 3: Definir variables para la generación de texto
num_ejemplos = 100  # Número de ejemplos de texto a generar

# Paso 4: Generar la base de datos con la API de OpenAI
prompt = "Escribe un trino sobre una reseña de un restaurante. El trino puede ser aleatoriamente bueno o malo sobre el restaurante"
base_datos = []
for _ in range(num_ejemplos):
    mensajes = [{"role": "user", "content": prompt}]
    respuesta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Modelo de OpenAI utilizado
        messages=mensajes,
        max_tokens=50,  # Longitud máxima de la respuesta
        temperature=1.5,  # Valor de temperatura para controlar la aleatoriedad
        n=1,  # Número de respuestas a generar
    )
    texto = respuesta["choices"][0]["message"]
    contenido_mensaje = texto["content"]
    base_datos.append(contenido_mensaje)

# Paso 5: Definir la instrucción de etiquetado
instruccion_etiquetado = "Clasifica el texto según las siguientes categorías de tono: Bueno, Malo. Solo responde con base a estas categorías, sin texto adicional ni puntos o espacios. Y solo se puede elegir una categoria"

# Paso 6: Crear la columna 'label' utilizando la API de OpenAI para el etiquetado
etiquetas = []
for texto in base_datos:
    mensajes = [
        {"role": "system", "content": instruccion_etiquetado},
        {"role": "user", "content": texto}
    ]
    respuesta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        max_tokens=150,
        temperature=0.5,
        n=1,
        stop=None,
    )["choices"][0]["message"]["content"]
    etiquetas.append(respuesta)

# Paso 7: Combinar el texto y las etiquetas en un DataFrame
df = pd.DataFrame({"text": base_datos, "label": etiquetas})

# Paso 8: Guardar el DataFrame en un archivo CSV
df.to_csv("train.csv", index=False)

# Paso 9: Mostrar el DataFrame resultante
df


**Datos propios**

También se puede usar un data set con datos propios. Es cuestión de seguir los siguientes pasos:

1. Abrir Excel
2. Crear un columna llamada "text" y otra "label"
3. En la columna "text" poner los ejemplos de textos que se quieran clasificar
4. En la columna "label" poner la respectiva etiqueta que aplique para cada texto.
5. Guardar el archivo en formato .csv. Se recomienda guardarlo bajo el nombre train para que el siguiente código aplique a los datos propios.
6. Subir los datos a Colba: ir al símbolo de la carpeta y en elegir el emogi de la hoja con la flecha dentro.







In [ ]:
# Paso 10: Convertir las etiquetas a formato numérico
# Mapear las clases a números
etiquetas_numericas = {"Bueno": 0, "Malo": 1}
etiquetas_numericas_convertidas = [etiquetas_numericas[etiqueta] for etiqueta in etiquetas]

# Combinar el texto y las etiquetas numéricas en un nuevo DataFrame
nuevo_df = pd.DataFrame({"text": base_datos, "label": etiquetas_numericas_convertidas})

# Guardar el nuevo DataFrame en un archivo CSV
nuevo_df.to_csv("train_numerico.csv", index=False)

# Mostrar el nuevo DataFrame resultante
nuevo_df

# Entrenamiento

In [ ]:
# Paso 11: Instalar los paquetes necesarios para el entrenamiento del modelo
!pip install transformers datasets  # Instalar paquetes transformers y datasets
!pip install transformers[torch]  # Instalar transformers con soporte para PyTorch
!pip install accelerate  # Instalar accelerate
!pip install evaluate  # Instalar evaluate

import pandas as pd
from datasets import load_dataset, Dataset  # Importar Dataset de datasets
from transformers import AutoTokenizer  # Importar AutoTokenizer de transformers
from transformers import AutoModelForSequenceClassification  # Importar AutoModelForSequenceClassification de transformers
from transformers import TrainingArguments, Trainer  # Importar TrainingArguments y Trainer de transformers
import numpy as np  # Importar numpy
import evaluate  # Importar evaluate

# Paso 12: Cargar el conjunto de datos desde el archivo CSV
dataset = pd.read_csv("train_numerico.csv")

# Paso 13: Dividir el conjunto de datos en conjuntos de entrenamiento y evaluación
from sklearn.model_selection import train_test_split

# Asumiendo que tienes columnas llamadas "text" y "label"
train_texts, eval_texts, train_labels, eval_labels = train_test_split(dataset["text"], dataset["label"], test_size=0.2, random_state=42)

# Crear diccionarios para los conjuntos de entrenamiento y evaluación
train_dataset = {"text": train_texts, "label": train_labels}
eval_dataset = {"text": eval_texts, "label": eval_labels}

# Convertir diccionarios a objetos Dataset
train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

# Paso 14: Cargar el tokenizador BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # Cargar tokenizador BERT

# Paso 15: Definir la función para tokenizar
def tokenize_function(examples):  # Definir función para tokenizar
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Paso 16: Aplicar tokenización al conjunto de datos
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de entrenamiento
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de evaluación

# Paso 17: Agregar las etiquetas al conjunto de datos tokenizado
tokenized_train_dataset = tokenized_train_dataset.add_column("labels", train_labels)
tokenized_eval_dataset = tokenized_eval_dataset.add_column("labels", eval_labels)

# Paso 18: Cargar el modelo BERT para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=len(dataset["label"].unique()))  # Cargar modelo BERT para clasificación de secuencias

# Paso 19: Definir hiperparámetros de entrenamiento
training_args = TrainingArguments(output_dir="restaurants-reviews", evaluation_strategy="epoch")  # Definir hiperparámetros de entrenamiento con evaluación por época

# Paso 20: Cargar la métrica de precisión
metric = evaluate.load("accuracy")  # Cargar métrica de precisión

# Paso 21: Definir la función para calcular métricas
def compute_metrics(eval_pred):  # Definir función para calcular métricas
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



In [ ]:
# Paso 22: Crear el objeto Trainer (continuación)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)  # Crear objeto Trainer

# Paso 23: Entrenar el modelo
trainer.train()  # Entrenar el modelo

# Probar el modelo

In [ ]:
# Paso 24: Importar el pipeline de transformers
from transformers import pipeline  # Importar pipeline de transformers

# Paso 25: Definir un prompt de ejemplo
prompt = "food is very very good, and perfect service"

# Paso 26: Crear el pipeline de clasificación de texto
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)  # Crear pipeline de clasificación de texto

# Paso 27: Realizar inferencia con el texto de ejemplo
result = pipe(f"<s>[INST] {prompt} [/INST] \n\n")  # Realizar inferencia con texto de ejemplo

# Paso 28: Imprimir el resultado
print(result)  # Imprimir resultado



In [ ]:
# Paso 29: Mejorar la respuesta del modelo

# Diccionario para convertir etiquetas numéricas a etiquetas originales
etiquetas_originales = {"LABEL_0": "Bueno", "LABEL_1": "Malo"}

# Paso 30: Definir la función para convertir predicciones
def convertir_predicciones(predicciones):
    etiquetas_predichas = []
    for prediccion in predicciones:
        etiqueta_predicha = None
        for clave, valor in etiquetas_originales.items():
            if clave == prediccion:
                etiqueta_predicha = valor
                break
        if etiqueta_predicha is None:
            etiqueta_predicha = prediccion
        etiquetas_predichas.append(etiqueta_predicha)
    return etiquetas_predichas

In [ ]:
# Paso 31: Ejemplo de inferencia y conversión de predicciones
prompt = "food is very very nice"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)
result = pipe(prompt)[0]
prediccion_numerica = result['label']
etiqueta_predicha = convertir_predicciones([prediccion_numerica])[0]
print(f"Predicción: {etiqueta_predicha}")

# Subir el modelo a HuggingFace

In [ ]:
# Paso 32: Importar la función notebook_login de huggingface_hub
from huggingface_hub import notebook_login  # Importar notebook_login de huggingface_hub

# Paso 33: Iniciar sesión en Hugging Face Hub
notebook_login()  # Iniciar sesión en Hugging Face Hub



In [ ]:
# Paso 34: Publicar el modelo en Hugging Face Hub
trainer.push_to_hub()  # Publicar modelo en Hugging Face Hub